# Woolsey Fire Model

The model defined in this notebook seeks to model how pre-fire conditions—including vegetation, topography, and climate—contributed to burn severity during the Woolsey Fire. The configuration of the model can be modified under the **Configure model** section, and the results are saved to the **models** directory in the **woolsey_fire** folder when the run finishes. 

This notebook is based on an example developed by Leah Wasser.

In [ ]:
import csv
from datetime import datetime
import json
import os
import pickle
import re
import warnings

from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, SMOTENC
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
from scipy.stats import mode
from skimage.exposure import rescale_intensity
from skimage.filters import gaussian
from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier
)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    classification_report,
    f1_score,
    mean_squared_error
)
import xarray as xr

from ea_drought_burn.config import DATA_DIR
from ea_drought_burn.notebooks import run_notebook
from ea_drought_burn.utils import (
    aggregate,
    agg_to_raster,
    copy_xr_metadata,
    create_sampling_mask,
    draw_legend,
    plot_bands,
)




# Constants used when choosing a sampler
DO_NOT_BALANCE = 1
BALANCE_USING_CLASS_WEIGHT = 2




# Set working directory to the earthpy data directory
os.chdir(os.path.join(DATA_DIR, "woolsey-fire"))

# Set base plot style
sns.set(font_scale=1.2, style="white")
plt.rc("figure.constrained_layout", use=True, h_pad=15/72, w_pad=15/72)

# Disable interpolation to prevent NaN erosion
plt.rc("image", interpolation=None)

In [ ]:
def float_format(num):
    """Formats float to a string for output by pandas
    
    Parameters
    ----------
    num: int or float
        number to format for print
        
    Returns
    -------
    str
        number as a string
    """
    return f"{num:.3f}" if num % 1 else str(int(num))


def one_line_report(report, labels, metrics):
    """Converts a classification report to a dict
    
    Parameters
    ----------
    report: str
        output from `sklearn.metrics.classification_report`
    labels: list of str
        labels targeted by classifier
    metrics: list of str
        columns from the classification report to capture. Metric names
        are shortened to 2-3 character abbreviations in the output.
        
    Returns
    -------
    dict
        dict containing key metrics for each class from the classification
        report
    """

    lines = iter(report.splitlines())
    
    # Create ordered list of column headings
    cols = ["accuracy", "pre_macro_avg", "rec_macro_avg", "f1_macro_avg"]
    for label in labels:
        for metric in metrics:
            cols.append(f"{metric}_{label}")
    
    # Shorten keys to 2-3 letter abbreviations
    keys = ["label"] + [k[:3].rstrip("-") for k in next(lines).split()]
    
    one_line = {}
    for line in lines:
        if line:
            
            # Convert strings to floats
            vals = [
                float(s) if s.isnumeric() else s 
                for s in re.split(r"  +", line.strip())
            ]
            
            # Map values to keys
            if len(vals) == len(keys):
                for key, val in dict(zip(keys, vals)).items():
                    one_line[f"{key}_{vals[0].replace(' ', '_')}"] = val
            elif vals[0] == "accuracy":
                one_line[vals[0]] = vals[-2]
            elif vals[0] == "macro avg":
                one_line["pre_macro_avg"]= vals[1]
                one_line["rec_macro_avg"]= vals[2]
                one_line["f1_macro_avg"]= vals[3]
    
    return {c: one_line.get(c, "") for c in cols}


def prep_for_model(x, y, mask=None):
    """Prepares data for use in a sklearn model
    
    Parameters
    ----------
    x: xarray.DataArray
        an array containing the explanatory data. Can contain more than one
        band.
    y: xarray.DataArray
        an array with a single band containing the response variable
    mask: numpy.array
        an array used to mask x and y
    
    Returns
    -------
    tuple of xarray.DataArray
        x and y formatted for use in a sklean model
    """
    
    x = x.copy()
    y = y.copy()
    
    # Ignore pixels set to False in the mask
    if mask is not None:
        x = x.where(mask)
        y = y.where(mask)

    # Convert to 1D arrays
    x = np.array([np.ravel(b) for b in x])
    y = np.ravel(y)

    # Limit to values that are finite in all arrays
    xy_mask = y.copy()
    for band in x:
        xy_mask *= band

    x = np.array([b[np.isfinite(xy_mask)] for b in x])
    y = y[np.isfinite(xy_mask)]
    
    return x.transpose(), y


def slugify(val):
    """Makes a string suitable for a filename
    
    Parameters
    ----------
    val: str
        the string to base the filename on
        
    Returns
    -------
    str
        the string as a filename
    """
    return re.sub(r"[^a-z0-9]+", "_", val.lower()).strip("_")

## Restore and prepare data

To ensure consitency across notebooks and avoid repeating code, all data is loaded using **1-load_data.ipynb** and restored as needed in individual notebooks. Please see that notebook for details about the sources of data used here.

In [ ]:
# Check for stored ready variable and run load-data.ipynb if not found
%store -r woolsey_data_ready
try:
    woolsey_data_ready
except NameError:
    print("Running load-data.ipynb...")
    run_notebook("1-load-data.ipynb")

# Retore variables using storemagic. Each variable is restored explicitly to
# avoid confusion about where variable names are coming from.
%store -r all_data
%store -r reproj_to
%store -r res

%store -r hill_fire
%store -r woolsey_fire
%store -r hill_and_woolsey_fires

%store -r cmap_dnbr
%store -r labels_dnbr
%store -r cmap_vegetation
%store -r labels_vegetation
%store -r prism_grid

# Set crop and fire boundaries
fire_name = "Woolsey Fire"
fire_bound = {"Woolsey Fire": woolsey_fire, "Hill Fire": hill_fire}[fire_name]
crop_bound = fire_bound.envelope

# Create dict to store figures
figures = {}

# Set datetime param for model run
run_datetime = datetime.now().strftime("%Y%m%dT%H%M%S")

### Aggregate data and define model-only fields

Model-only fields (like the blurred views of vegetation communities) are generally subsets or calculations that seemed like they might be useful for the model but didn't really work out and weren't interesting/relevant enough to add to **load-data.ipynb**.

In [ ]:
# Create lookup for all data
datasets = all_data.copy()

# Extract last year for fraction-alive data and derivatives
fal = datasets["FAL (2013-2016)"]
datasets["FAL (2016)"] = fal[-1]
datasets["dFAL (2013-2016)"] = fal[0] - fal[3]
datasets["Dead (2016)"] = datasets["Dead (2013-2016)"][-1]

# Group communities into grass/shrubs and trees
comm = datasets["Community"]
shrubs_trees = comm.copy()
shrubs_trees = xr.where(shrubs_trees == 2, 1, shrubs_trees)
shrubs_trees = xr.where(shrubs_trees == 3, 1, shrubs_trees)
shrubs_trees = xr.where(shrubs_trees == 4, 2, shrubs_trees)
shrubs_trees = xr.where(shrubs_trees == 5, 2, shrubs_trees)
shrubs_trees = shrubs_trees.where((shrubs_trees == 1) | (shrubs_trees == 2))

shrubs_trees = shrubs_trees.rio.write_crs(datasets["Community"].rio.crs)
datasets["Community (Shrubs and Trees)"] = shrubs_trees

# Create blurred community maps. The hope here was that the blurred views
# would help the model make a stronger connection between oak woodlands and
# burn severity by getting near-oak pixels involved. Didn't really work.
for i, name in enumerate([
    "Annual Grass",
    "Chaparral",
    "Coastal Sage Shrub",
    "Oak Woodland",
    "Riparian",
    "Substrate"
]):
    subset = xr.where(comm == i + 1, 1, 0)
    blurred = gaussian(
        subset.where(np.isfinite(subset), 0), sigma=3, truncate=6
    )
    blurred = rescale_intensity(blurred, out_range=(0, 1))
    blurred = copy_xr_metadata(comm, blurred)
    datasets[name] = blurred
    
subset = xr.where((comm == 4) | (comm == 5), 1, 0)
blurred = gaussian(subset.where(np.isfinite(subset), 0), sigma=3, truncate=6)
blurred = rescale_intensity(blurred, out_range=(0, 1))
blurred = copy_xr_metadata(comm, blurred)
datasets["Trees"] = blurred

# Use the mean for each pixel for each four-year set of climate data
for key in (
    "Days Precipitation (2013-2016)",
    "Max VPD (2013-2016)",
    "Minimum Temperature (2013-2016)",
    "Heat Days Over 95 (2013-2016)",
    "Cumulative Precipitation (2013-2016)"
):
    datasets[key] = datasets[key].sum(axis=0)

### Configure model parameters

The constants in the cell below—`FEATURES`, `LABELS`, `CATEGORIES`, `CLASSIFIER`, `CLASSIFIER_PARAMS`, `SAMPLING PARAMS`, `BALANCE_DATA`, `DRAW_PAIR_PLOTS`, and `RUN_NAME`—are used to configure how the model will run. At the end of the notebook, the notebook saves the content of each variable along with the results of the model. Note that `fire_name` and `fire_bound` are set above to maintain consistency with other notebooks in this project. 

Note that the model presented here only considers pre-fire conditions. Conditions (like wind) and interventions (like the firefighting response) are not included even though they contributed to how the fire developed.

In [ ]:
# Select features by unhashing lines with strings
FEATURES = [
    # Vegetation
    "Community",
    #"Community (Shrubs and Trees)",
    #"Annual Grass",
    #"Chaparral",
    #"Coastal Sage Shrub",
    #"Oak Woodland",
    #"Riparian",
    #"Substrate",
    #"Trees",
    
    # Vegetation health
    "FAL (2016)",
    #"Dead (2016)",
    #"dFAL (2013-2016)",
    #"Years Dead",
    #"Live FAL",
    "Last Burned",
        
    # Pre-fire spectral indices
    #"LFMC (2018)",
    #"NDVI (2018)",
    #"NDMI (2018)",
    #"NDWI (2018)",
    #"SAVI (2018)",
    
    # Topography
    "Elevation",
    #"Aspect",
    "Folded Aspect",
    "Slope",
    
    # Climate data aggregated to water year (Oct 1-Sep 30) for 2013-2016
    #"Days Precipitation (2013-2016)",
    #"Max VPD (2013-2016)",
    #"Minimum Temperature (2013-2016)",
    #"Heat Days Over 95 (2013-2016)",
    #"Cumulative Precipitation (2013-2016)",
    
    # Climate data aggregated to calendar year for 2018
    "Precipitation (2018)",
    #"Mean Dew Point Temperature (2018)",
    #"Maximum Temperature (2018)",
    #"Mean Temperature (2018)",
    #"Minimum Temperature (2018)",
    #"Maximum VPD (2018)",
    #"Minimum VPD (2018)",
    
    # Burn severity
    #"MTBS dNBR",
    #"MTBS Classified dNBR",
    #"MTBS Classified Burned/Unburned",
    #"Sentinel-2 dNBR",
    #"Sentinel-2 Classified dNBR",
]

# Response variable. Must be one of the classified dNBR plots. The MTBS
# version is preferred.
LABELS = "MTBS Classified dNBR"

# Non-continuous features are handled differently by some sampling algorithms,
# so make a list of any categorical data that may be considered by the model.
CATEGORIES = [
    "Community",
    "Community (Shrubs and Trees)",
    "Dead (2016)",
    "Years Dead",
    "Last Burned",
    "MTBS Classified dNBR",
    "MTBS Classified Burned/Unburned",
    "Sentinel-2 Classified dNBR"
]

# Classifier class. RandomForestClassifier and GradientBoostingClassifier
# are imported above, but really any scikit-learn classifier could be used.
CLASSIFIER = RandomForestClassifier

# Keyword arguments passed to classifier. If this dict does not include 
# n_estimators, the notebook will tune the classifier based on a set of
# reasonable parameters using the GridSearchCV function. Tuning is slow, so
# provide params if you can.
CLASSIFIER_PARAMS = {
    
    # RandomForestClassifer params. The class_weight parameter is ignored
    # if specified. Set BALANCE_DATA below to True if you want to rebalance
    # imbalanced data.
    "oob_score": True,
    "n_estimators": 200,
    "max_depth": 5 
    
    # GradientBoostingClassifier params
    #"n_estimators": 100,
    #"learning_rate": 1.0,
    #"max_depth": 1,
    #"random_state": 0

}

# Keyword arguments pass to create_sampling_mask
SAMPLING_PARAMS = {
    "counts": {"training": 9000, "validation": 3000},
    "seed": 0
}

# Specifies whether to under/over-sample data to better model imbalanced data.
# See "Handled imbalanced data" for more info on this topic.
BALANCE_DATA = True

# Specifies whether to draw pair plots. Drawing the plots is time-consuming, 
# so set to False if you want to run a bunch of different models.
DRAW_PAIR_PLOTS = False

### Name run and set mask

In [ ]:
# Assign this run a name. If a subset, make sure the mask is correct.
RUN_NAME = "All Vegetation"
if BALANCE_DATA:
    RUN_NAME += " (Balanced)"

# Limit the model to a subset of the available pixels. This should be
# understood as training a subset-specific model (as opposed to training a
# model on a subset that can then be applied to the whole dataset). 
#
# The hashed lines correspond to:
# 
# 1. Selecting a specific community
# 2. Selecting all grass/shrubs
# 3. Selecting all trees
cond = (
    (datasets[LABELS] < 5)
    #& (datasets["Community"] == 4)
    #& ((datasets["Community"] == 1) | (datasets["Community"] == 2) | (datasets["Community"] == 3))
    #& ((datasets["Community"] == 4) | (datasets["Community"] == 5))  3 trees
)

mask = xr.where(cond, True, False)
mask = mask.rio.clip(fire_bound.geometry, drop=False)
mask = mask.values

### Re-use an existing model

In [ ]:
# Re-use an existing model. This allows you to try a model on another fire. If
# model_dir is empty, a new model will be created below.
model_dir = None
if model_dir:
    # Get path to pickled model
    model_path = os.path.join(model_dir, "model.pickle")
    
    # Update parameters to match model. Note that some parameters, notably the
    # sampling mask, are not captured.
    with open(os.path.join(model_dir, "model.json")) as f:
        params = json.load(f)
    
    sampler_name = params["sampler"]
        
    FEATURES = params["features"]
    LABELS = params["labels"]
    CLASSIFIER_PARAMS = params["params"]["classifier_params"]
    SAMPLING_PARAMS = params["params"]["sampling_params"]

In [ ]:
# Create and verify the sampling mask
xda = reproj_to.rio.clip(fire_bound.geometry, drop=False)
while True:
    try:
        sampling_mask = create_sampling_mask(xda.copy().where(mask), **SAMPLING_PARAMS)
        break
    except ValueError:
        # Too many pixels in sample. Adjust counts down until there are enough.
        print(SAMPLING_PARAMS["counts"])
        for key, count in SAMPLING_PARAMS["counts"].items():
            SAMPLING_PARAMS["counts"][key] = int(count * 0.9)

# Verify that the sampling mask is the right shape
if sampling_mask.shape[-2:] != prism_grid.shape[-2:]:
    raise ValueError("Invalid shape")

In [ ]:
# Create training and validation subsets
training = {k: v.where(sampling_mask[0]) for k, v in datasets.items()}
validation = {k: v.where(sampling_mask[1]) for k, v in datasets.items()}

# Mask datasets
datasets = {k: v.where(mask) for k, v in datasets.items()}

# Create subset clipped to the fire boundary
clipped = {
    k: v.rio.clip(fire_bound.geometry, drop=False)
    for k, v in datasets.items()
}

# Create lookup for all subsets
subsets = {
    "all": datasets,
    "clipped": clipped,
    "training": training,
    "validation": validation,
}

### Plot sampling mask as a map

Plot a map view of the training and validation datasets. This plot mostly just confirms that the mask is behaving correctly and only pulling pixels from inside the fire scar. If the script balances the data when the sampling mask is created, you can see where pixels are being over- or under-sampled.

In [ ]:
# Plot the sampling mask
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
plot_bands(sampling_mask[0], ax=ax1, cbar=None, title=f"{RUN_NAME} - Training")
plot_bands(sampling_mask[1], ax=ax2, cbar=None, title=f"{RUN_NAME} - Validation")
for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, facecolor="none", edgecolor="gray", linewidth=1)

# Store figure to save later
figures["sampling_masks"] = fig

## Prepare and run the classifier

In [ ]:
# Create feature dataset based on keys selected near the top of the notebook
features = sorted([f for f in FEATURES if f != LABELS])

xdata = {}
for subset, lookup in subsets.items():
    
    bands = []
    for key in features:
        band = lookup[key]
        
        # If more than one layer, use the last one. You can get around this
        # behavior by selecting a layer or aggregating all layers above (for
        # example, climate data uses the mean of the four years).
        if len(band.shape) > 2:
            band = band[-1]
        
        bands.append(band)
        
        try:
            del bands[-1]["band"]
        except KeyError:
            pass

        bands[-1] = bands[-1].squeeze()
        bands[-1]["band"] = len(bands)
      
    xdata[subset] = xr.concat(bands, dim="band")

In [ ]:
# Create training and validation splits
tx, ty = prep_for_model(xdata["training"], training[LABELS])
vx, vy = prep_for_model(xdata["validation"], validation[LABELS])

### Plot histograms

Plot histograms of all data and subsets. If the script balances the data when the sampling mask is created, you can these plots to verify that the bins are all the same size (but as of now that behavior is no longer supported).

In [ ]:
# Use the same colors defined in the MTBS dNBR raster. These are defined in
# a color map defined in load-data.ipynb, but there is no obvious way to
# convert that map back to a list.
dnbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]

# Save histograms of labels as PNG
response = np.array([a[LABELS] for a in [clipped, training, validation]])

fig, axes = plt.subplots(3, 1, figsize=(8, 16))
for ax, arr, title in (zip(axes, response, ["All", "Training", "Validation"])):
    
    counts = {}
    for val in np.unique(arr[np.isfinite(arr)]):
        label = labels_dnbr[int(val) - 1].replace(" ", "\n", 1)
        counts[label] = np.sum(arr == val)
    
    ax.bar(counts.keys(), counts.values(), color=dnbr_colors)
    ax.set(title=f"{RUN_NAME} - {title}", ylabel="counts")
    
    # Rotate labels
    for label in ax.get_xticklabels():
        label.set_rotation(90) 

# Store figure to save later
figures["sampling_histograms"] = fig

### Tune the classifier

Each of the `scikit-learn` classifiers has a boatload of hyperparameters, some more mysterious than others. The
`GridSearchCV` methods tunes the hyperparameters using lists of possible values supplied by the user.

This operation is expensive, so it only runs if the user does not supply CLASSFIER_PARAMS above.

In [ ]:
# Tune hyperparameters if estimators, depth, etc. not given
CLASSIFIER_PARAMS.setdefault("random_state", 0)

if "n_estimators" not in CLASSIFIER_PARAMS:
    
    # This is a simplistic tuning, but the upside is that it should work with
    # either RandomForestClassifier or GradientBoostingClassifier.
    param_grid = {
        "max_depth": [4, 8, 16, None],
        "n_estimators": [100, 200, 500],
    }

    grid_search = GridSearchCV(
        estimator=CLASSIFIER(),
        param_grid=param_grid,
        scoring="f1_samples",
        n_jobs=-1,
        cv=5,
        return_train_score=True
    )
    grid_search.fit(tx, ty)
    
    # Update the params being passed to the classifier. You can also copy
    # these to the 
    CLASSIFIER_PARAMS.update(grid_search.best_params_)
    
    display(CLASSIFIER_PARAMS)

### Handle imbalanced data

High burn severities occurred in only a small part of the Woolsey Fire. The imbalanced data poses a challenge for the random-forest model, which happily ignores the low-frequency pixels because those misses make only a small contribution to the overall accuracy of the model. When the model is trained using the original data frequencies, it fails to predict any high-severity pixels at all.

Python machine-learning packages have implemented tools for working with imbalanced data, so there are options to address imbalance. The `RandomForestClassifier` in scikit-learn provides a keyword argument, `class_weight`, that allows users to rebalance data, while the imablanced-learn module provides a suite of different under- and oversampling functions to accomplish something similar. The results of early runs of the random forest model using `class_weight` weren't great, so I added the imbalanced-learn samplers to try to improve them. The current version tries out all the relevant samplers and keeps the one with the lowest F1 score.

The imbalanced-learn samplers use two algorithms--the Adaptive Synthetic (ADASYN) algortim and the Synthetic Minority Over-sampling Technique (SMOTE)--to generate synthetic samples to fill out low-frequency groups. That package also implements variants on the SMOTE algorithm that combine under- and over-sampling techniques (like SMOTEENN and SMOTETomek) or are suitable for a mix of categorical and continuous data (SMOTENC). The [documentation for imbalanced-learn](https://imbalanced-learn.org/dev/introduction.html) has more info about the different samplers, including comparisons between them.

I have not tried to tune the samplers, but in general, `class_weight` performs about as well as any of them.


#### Evaluating models based on imbalanced data

Precision, recall, and F1 scores for indivudal classes are useful for evaluating models trained using imbalanced data, which may have high overall accuracy while completely missing classes that appear infrequently. In scikit-learn, all three scores can be calculated using `sklearn.metrics.classification_report`. In imbalanced-learn, these plus a handful of other scores can be calculated using `imblearn.metrics.classification_report_imbalanced`.

When running a classifier, each class can be understood as its own binary system. A given pixel

1. is either part of that class or not
2. is either classified as that class or not

Within this framework, there are four possible outcomes for class A:

+ **True positive:** A is classified as A
+ **False positive/:** ~A is classified as A (error of commission)
+ **False negative:** A is classified as ~A (error of omission)
+ **True negative:** ~A is classified as ~A

These outcomes can be used to calculate precision and recall scores for each class:

+ **Precision** is calculated as **true positive / (true positive + false
  positive)** and is the percentage of pixels *classified as A* that are
  *actually in class A*. Precision is the same as user's accuracy. 


+ **Recall** is calculated as **true positive / (true positive + false negative)**
  and is the percentage of pixels *acutally in class A* that were *classified as
  A*. Recall is the same as producer's accuracy.


+ The **F1 score** combines precision and recall into a single metric by taking
  the harmonic mean of the two values: **2 * (precision * recall) / (precision +
  recall)**.

In [ ]:
# Run model. Use a loop so that the model can be run  multiple times to assess
# certain performance metrics, if desired. 
runs = []
for i in range(1):
    
    # Identify categorical features. Categories do not play well with some
    # sampling algorithms, so use different samplers if categories are usedS.
    cat_features = [i for i, f in enumerate(features) if f in CATEGORIES]
    
    # Both list of samplers contain two constants as the first two values in
    # the list tell the script to (1) not balance the data at all and (2) use
    # the class_weight argument in RandomForestClassifier to balance the data.
    if cat_features:
        samplers = [
            DO_NOT_BALANCE,
            BALANCE_USING_CLASS_WEIGHT,
            SMOTENC(categorical_features=cat_features, random_state=0)
        ]
    else:
        samplers = [
            DO_NOT_BALANCE,
            BALANCE_USING_CLASS_WEIGHT,
            ADASYN(sampling_strategy="all", random_state=0),
            SMOTE(random_state=0),
            SMOTEENN(random_state=0),
            SMOTETomek(random_state=0),
        ]
    
    # Try each sampler to see which one yields the highest f-score
    scored = {}
    for sampler in samplers:
        if model_dir:
            with open(model_path, "rb") as f:
                pipeline = pickle.load(f)
        elif sampler == DO_NOT_BALANCE:
            sampler_name = "unbalanced"
            pipeline = make_pipeline(CLASSIFIER(**CLASSIFIER_PARAMS))
        elif sampler == BALANCE_USING_CLASS_WEIGHT:
            # Skip class_weight if not using RandomForestClassifier
            if CLASSIFIER != RandomForestClassifier:
                continue
            sampler_name = "class_weight"
            params = CLASSIFIER_PARAMS.copy()
            params.update(class_weight="balanced")
            pipeline = make_pipeline(CLASSIFIER(**params))
        else:
            sampler_name = sampler.__class__.__name__
            pipeline = make_pipeline(sampler, CLASSIFIER(**CLASSIFIER_PARAMS))
        
        # Fit the model
        try:
            if not model_dir:
                pipeline.fit(tx, ty)
        except ValueError:
            pass
        else:
            # Rank models by the F1 score for high-severity fires
            observed = np.where(vy == 4, 1, 0)
            predicted = np.where(pipeline.predict(vx) == 4, 1, 0)
            score = f1_score(observed, predicted, average="binary")

            scored[score] = (sampler_name, pipeline)
        
        # The first entry in each list does not rebalance the data, so if
        # the BALANCE_DATA flag evaluates to False, the iterator stops here. 
        if not model_dir and not BALANCE_DATA:
            break
        
    # Use the classifier with the best results (here, the lowest F1 score)
    sampler_name, classifier = scored[sorted(scored.keys())[-1]]
    
    # Print results of the model. Suppress warnings about ill-defined labels.
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        
        # Limit labels to those present in the dataset
        labels = ["unburned", "low", "moderate", "high", "enhanced regrowth"]
        predicted = classifier.predict(vx)
        vals = set(list(np.unique(vy)) + list(np.unique(predicted)))
        labels = [lbl for i, lbl in enumerate(labels) if (i + 1) in vals]
        
        report = classification_report(
            vy, classifier.predict(vx), target_names=labels
        )
    
    print(f"Sampler: {sampler_name}")
    print(report)
    
    # Summarize the run as a list
    run = []
    
    # Get the OOB score if using RandomForestClassifier
    try:
        run.append(classifier.steps[-1][-1].oob_score_)
    except AttributeError:
        run.append(0)
    run.append(mean_squared_error(vy, classifier.predict(vx)) ** 0.5)
    
    # Add precision, recall, and related metrics extracted from report
    rpt = one_line_report(
        report, ["unburned", "low", "moderate", "high"], ["pre", "rec", "f1"]
    )
    run.extend(rpt.values())
    
    # Add feature importances
    run.extend(classifier.steps[-1][-1].feature_importances_)
    
    runs.append(run)

### Summarize and plot results

The model results include three metrics beyond the precision-and-recall suite defined above:

+ The **out-of-bag score** is a cross-validation metric calculated by
  RandomForestClassifier as it goes. Each tree created by the classifier uses
  only a subset of the available training data, and the out-of-bag score is
  calculated by comparing the predictions of trees trained with and without each
  subset. Lower values are better.


+ **Root mean square error** estimates the average standard deviation between
  predicted and observed values. Lower values are better.


+ **Feature importance** estimates how important each feature was to the
  predictions made by the model. Higher values indicate higher importance.

In [ ]:
# Show results from each run in a dataframe with many, many columns. The
# results of the last run are the ones that get stored at the end of the
# notebook.

# Get a list of feature columns
feat_cols = {f: slugify(f"feat_imp_{f}") for f in features}

cols = ["oob", "rmse"] + list(rpt) + list(feat_cols.values())
results = pd.DataFrame(runs, columns=cols)

# Map feature names to importances
feat_imp = {f: getattr(results.iloc[-1], c) for f, c in feat_cols.items()}

# Transpose results for display
with pd.option_context("display.max_columns", None):
    display(results.round(3).transpose())

In [ ]:
# Aggregate the results from the separate runs if multiple. No output from
# this cell if only one row.
if len(results) > 1:
    agg_results = results.agg(["mean", "std", "min", "max", "count"])
    
    # Transpose results for display
    with pd.option_context("display.max_columns", None):
        display(agg_results.round(3).transpose())

#### Confusion matrix

The confusion matrix illustrates how values predicted by the model compare to the observed values. The rows contain the predicted values and the columns the observed values for each class. The diagonal shows the number of pixels where predicted matches observed. The precision and recall scores described above summarize the information presented in the confusion matrix. The matrix itself is useful for evaluating which classes are being confused.

In [ ]:
# Create the confusion matrix
conf_mtx = pd.DataFrame({"observed": vy, "predicted": classifier.predict(vx)})

# Cross-tabulate predictions
crosstab = pd.crosstab(conf_mtx["observed"], conf_mtx["predicted"], margins=True)
crosstab

## Apply the model to the full dataset

In [ ]:
# Apply the trained model to the full dataset. Note that this calculation
# excludes any pixels that were excluded from the model.
all_x, all_y = prep_for_model(xdata["clipped"], subsets["clipped"][LABELS])
predicted = classifier.predict(all_x)

In [ ]:
# Create classification report based on the full dataset. Originally added
# to help assess datasets where oversampling was done via the sampling mask,
# now just here as a sanity check.

# Limit labels to those present in the dataset
labels = ["unburned", "low", "moderate", "high", "enhanced regrowth"]
vals = set(list(np.unique(all_y)) + list(np.unique(predicted)))
labels = [lbl for i, lbl in enumerate(labels) if (i + 1) in vals]

# Create classification report. Suppress warnings about ill-defined labels.
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")    
    report_all = classification_report(all_y, predicted, target_names=labels)
    
print(report_all)

### Plot model results as a map

In [ ]:
# Reload all x data. The predict method can't handle NaNs, but we can't drop 
# pixels like above because we need to reshape to the original array to plot.
# Set NaNs to -9999 instead.
all_x = np.array([np.ravel(x) for x in xdata["clipped"]])
all_x[~np.isfinite(all_x)] = -9999
predicted_ = classifier.predict(all_x.transpose())

In [ ]:
# Create a plottable versions of the observed and predicted data    
observed = subsets["clipped"][LABELS].copy()
observed = copy_xr_metadata(subsets["clipped"][LABELS], observed)
observed = observed.rio.clip(fire_bound.geometry)

predicted = predicted_.reshape(*subsets["clipped"][LABELS].shape)
predicted = copy_xr_metadata(subsets["clipped"][LABELS], predicted)
predicted = predicted.rio.clip(fire_bound.geometry)
predicted = predicted.where(np.isfinite(observed))

# Aggregate to PRISM grid. Use primarily if you need to compare burn
# severity to the aggregated pair plot but otherwise should be hashed.
#observed = agg_to_raster(observed, prism_grid).rio.write_crs(observed.rio.crs)
#predicted = agg_to_raster(predicted, prism_grid).rio.write_crs(predicted.rio.crs)

# Sort features based on importance
feat_sorted = [
    k for k, v in sorted(feat_imp.items(), key=lambda kv: -kv[1]) if v > 0.05    
]

# Set vmin and vmax so the whole range appears
vmin = 1
vmax = 5

# Set color map based on response variable in LABELS
cmap = cmap_dnbr
labels = labels_dnbr
classes = range(5)
if "Burned" in LABELS:
    cmap = ListedColormap(dnbr_colors[1:3])
    labels = ["Unburned/Low", "Moderate/High"]
    classes = range(2)

# Draw comparison plot. Note top five features
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
fig.suptitle(
    f"{RUN_NAME} - {CLASSIFIER.__name__} - {LABELS} - {res} m\n"
    f"Features: {' > '.join(feat_sorted)}"
)

plot_bands(observed,
           ax=ax1,
           title="Observed - MTBS",
           cmap=cmap,
           vmin=vmin,
           vmax=vmax,
           cbar=False,
           scale=False)

plot_bands(predicted,
           ax=ax2,
           title="Predicted",
           cmap=cmap,
           vmin=vmin,
           vmax=vmax,
           cbar=False,
           scale=False)

for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

# Position legend based on which fire is being plotted (Woolsey or Hill)
if fire_name == "Woolsey Fire":
    ax = ax1
    bbox = (0.01, 0.99)
elif fire_name == "Hill Fire":
    ax = ax2
    bbox=(0.71, 0.1)

# Draw legend
draw_legend(im_ax=ax.get_images()[0],
            classes=classes,
            titles=labels,
            bbox=bbox)

# Store figure to save later
figures["map"] = fig

## Save parameters and results

Each model run is saved in the **outputs/models/{run_datetime}**. High-level statistics from each run are saved to **outputs/models/results.csv** to make it possible to do quick comparisons between different runs.

In [ ]:
# Collect parameters in a dict
model = {
    "name": RUN_NAME,
    "classifier": CLASSIFIER.__name__,
    "sampler": None if sampler is None else sampler.__class__.__name__,
    "features": features,
    "labels": LABELS,
    "params": {
        "classifier_params": CLASSIFIER_PARAMS,
        "sampling_params": SAMPLING_PARAMS}
}

# Note existing model if used
if model_dir:
    model["model_dir"] = model_dir

# Set the run name to the date/time of the run
models_path = os.path.join("outputs", "models")
outdir = os.path.join(models_path, f"{slugify(RUN_NAME)}_{run_datetime}")

try:
    os.makedirs(outdir)
except OSError:
    pass

In [ ]:
# Pickle the model itself
with open(os.path.join(outdir, "model.pickle"), "wb") as f:
    pickle.dump(classifier, f)

# Save validation results as JSON
results_ = {"model_id": os.path.basename(outdir)}
results_.update({
    k: f"{float(v):.3f}" if v else "" 
    for k, v in results.iloc[-1].to_dict().items()
})
with open(os.path.join(outdir, "results.json"), "w", encoding="utf-8") as f:
    json.dump(results_, f, indent=2)
    
# And also as text
with open(os.path.join(outdir, "report.txt"), "w") as f:
    f.write(report)
    
# Save model parameters
with open(os.path.join(outdir, "model.json"), "w", encoding="utf-8") as f:
    json.dump(model, f, indent=2)

# Save figures
for fn, fig in figures.items():
    fig.savefig(os.path.join(outdir, f"{fn}.png"), bbox_inches="tight")
figures = {}

# Save crosstab and result tables as HTML
css = (
    "<style>"
    "table {border-collapse: collapse;}"
    "td, th { text-align: center; padding: 8px; }"
    "th { background-color: #eee; }"
    "</style>"
)

html = crosstab.transpose().to_html(float_format=float_format)
with open(os.path.join(outdir, "crosstab.htm"), "w") as f:
    f.write(css + "\n" + html)

try:
    html = agg_results.transpose().to_html(float_format=float_format)
except NameError:
    html = results.transpose().to_html(float_format=float_format)
with open(os.path.join(outdir, "results.htm"), "w") as f:
    f.write(css + "\n" + html)

In [ ]:
# Summarize results from all runs in models directory in a CSV. This is easier
# than adding models one at a time because fields may differ between runs.
results = []
for root, dirs, files in os.walk(models_path):
    for fn in files:
        if fn == "results.json":
            with open(os.path.join(root, fn)) as f:
                results.append(json.load(f))

results.sort(key=len, reverse=True)
keys = {}
for result in results:
    keys.update(result)
keys = list(keys)
keys = (
    [k for k in keys if not k.startswith("feat_")] + \
    sorted([k for k in keys if k.startswith("feat_")])
)
    
path = os.path.join(models_path, "results.csv")
with open(path, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f, dialect="excel")
    writer.writerow(keys)
    for row in sorted(results, key=lambda r: r["model_id"]):
        writer.writerow([row.get(k, "") for k in keys])


## Evaluate source data using pair plots

Pair plots can be time-consuming to generate and can offer insight into areas where the model went wrong, so I've saved them for last. Use the pair plots below to look for:

1. **Collinear features.** Use this plot to assess whether to remove a given
   feature from the model.
2. **Clusters of misclassified features.** Use these plots to  work out why and
   how pixels are being misclassified.  
   
Pair plots are also saved to the **models** folder for the current run.

The first pair plot shows the source data for the model aggreated to the PRISM grid (4 km blocks). This helps clear out some of the noise in the point data and aligns the high-resolution data to the climate data.

In [ ]:
pairs = []

# For the first plot, aggreagate the original data to the PRISM grid before
# plotting. This reduces the noise seen on the point plots. Use the
# unclassified version of the label for all aggregations except mode.

func = np.nanmean  # for mode: lambda a: mode(a).mode

yname = LABELS
kind = "mode"
if func.__name__ != "<lambda>":
    yname = LABELS.replace("Classified ", "")
    kind = "mean"

x = xdata["clipped"]
y = subsets["clipped"][yname]

# Aggregate each array to the PRISM grid and append
x = np.array([np.ravel(aggregate(b, prism_grid, func=func)) for b in x]).transpose()
y = np.ravel(aggregate(y, prism_grid, func=func))

pairs.append((x, y, yname, f"All source data aggregated to PRISM grid ({kind})"))

The rest of the pair plots look at pixels that have been misclassified. The burn-severity classes overlap significantly, and these plots have mostly not been useful for developing strategies to separate them.

In [ ]:
# Limit output to misclassified pixels
misses = {
    "unburned": predicted == 1,
    "low severity": predicted == 2,
    "moderate severity": predicted == 3,
    "high severity": predicted == 4
}

for key, missed in misses.items():
    mask_ = mask * (observed != predicted) * missed
    x, y = prep_for_model(xdata["training"], subsets["training"][LABELS], mask_)
    
    # Add the pairs only if there is any data to plot
    if np.isfinite(y).any():
        pairs.append(
            (x, y, f"Classified dNBR", f"Pixels misclassified as {key}")
        )

In [ ]:
# Draw pair plots if desired
if DRAW_PAIR_PLOTS:
    
    # Pair plots include yellow glyphs, so use a black background.
    sns.set(style="ticks", context="talk")
    plt.style.use("dark_background")

    # Draw pair plots
    for x, y, yname, title in pairs:

        paired = pd.DataFrame(data=x, columns=features)
        paired[yname] = y

        # Suppress warnings from pairplot, which can throw a lot of warnings
        # depending on the source data.
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")

            # Define palette based on whether response is categorical 
            # or continuous
            if "Classified" in yname:
                palette = [
                    dnbr_colors[int(i - 1)] for i
                    in np.unique(y[np.isfinite(y)])
                ]  
            else:
                palette = "RdYlGn_r"

            grid = sns.pairplot(paired,
                                height=5,
                                plot_kws={"s": 200},
                                hue=yname,
                                palette=palette)

            grid.fig.suptitle(title, y=1.02)

            # The output dir is set now, so save the figure
            grid.fig.savefig(os.path.join(outdir, f"pp_{slugify(title)}.png"),
                             bbox_inches="tight")

    # Reset styles
    sns.set(font_scale=1.5, style="white")
    plt.rcParams.update(plt.rcParamsDefault)